In [1]:
#Import libraries
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output, State
import torch
from transformers import GPT2LMHeadModel, GPT2Tokenizer
from tqdm import tqdm, trange
import torch.nn.functional as F

In [2]:
#Import tokenizer and model
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
device=torch.device('cpu')
model = torch.load('model.pt', map_location=device)
model_pop = torch.load('model_pop.pt', map_location=device)

In [3]:
def generate(
    model,
    tokenizer,
    prompt,
    entry_count=10,
    entry_length=30, #maximum number of words
    top_p=0.8,
    temperature=1.,
):

    model.eval()

    generated_num = 0
    generated_list = []

    filter_value = -float("Inf")

    with torch.no_grad():

        for entry_idx in trange(entry_count):

            entry_finished = False

            generated = torch.tensor(tokenizer.encode(prompt)).unsqueeze(0)

            for i in range(entry_length):
                outputs = model(generated, labels=generated)
                loss, logits = outputs[:2]
                logits = logits[:, -1, :] / (temperature if temperature > 0 else 1.0)

                sorted_logits, sorted_indices = torch.sort(logits, descending=True)
                cumulative_probs = torch.cumsum(F.softmax(sorted_logits, dim=-1), dim=-1)

                sorted_indices_to_remove = cumulative_probs > top_p
                sorted_indices_to_remove[..., 1:] = sorted_indices_to_remove[
                    ..., :-1
                ].clone()
                sorted_indices_to_remove[..., 0] = 0

                indices_to_remove = sorted_indices[sorted_indices_to_remove]
                logits[:, indices_to_remove] = filter_value

                next_token = torch.multinomial(F.softmax(logits, dim=-1), num_samples=1)
                generated = torch.cat((generated, next_token), dim=1)

                if next_token in tokenizer.encode("<|endoftext|>"):
                    entry_finished = True

                if entry_finished:

                    generated_num = generated_num + 1

                    output_list = list(generated.squeeze().numpy())
                    output_text = tokenizer.decode(output_list)
                    generated_list.append(output_text)
                    break
            
            if not entry_finished:
              output_list = list(generated.squeeze().numpy())
              output_text = f"{tokenizer.decode(output_list)}<|endoftext|>" 
              generated_list.append(output_text)
                
    return generated_list

In [4]:
#START OF THE ACTUAL APP INTERFACE
app = dash.Dash()

#Create the layout of the app
app.layout = html.Div([
    
 html.H1(children='LYRICS GENERATOR', style={'textAlign': 'center'
        }),
    
#Add dropdown menu for type of music
dcc.Dropdown(id='music_type',
    options=[
        {'label': 'Rock', 'value': 'Rock'},
        {'label': 'Pop', 'value': 'Pop'}
    ],
    value='Rock'),

#Add place where text would be inserted    
  dcc.Textarea(
        id='textarea-state-example',
        value='',
        style={'width': '100%', 'height': 200, 'position':'absolute', 'top':'130px'},
    ),
    html.Button('Submit', id='textarea-state-example-button', n_clicks=0, 
               style={'position':'absolute', 'left':'800px', 'top':'350px'}),
    
    html.Div(id='textarea-state-example-output', style={'whiteSpace': 'pre-line', 'position':'absolute', 
                        'left':'30px', 'top':'400px', 'font-size': 'large','height': '200px', 'width': '700px'})
    
])  

@app.callback(
    Output('textarea-state-example-output', 'children'),
    [Input('textarea-state-example-button', 'n_clicks'), Input('music_type', 'value')],
    State('textarea-state-example', 'value')
)

def update_output(n_clicks, value, value2):
    
    if n_clicks > 0:
        if value == 'Rock':
            generated = generate(model.to('cpu'), tokenizer, value2, entry_count=1)
            
            #Clean the output
            generated2 = ' '.join(generated)
            to_remove = generated2.split('.')[-1]
            
            my_text = generated2.replace(to_remove,'')
        
        if value == 'Pop':
            generated = generate(model_pop.to('cpu'), tokenizer, value2, entry_count=1)
            
            #Clean the output
            generated2 = ' '.join(generated)
            to_remove = generated2.split('.')[-1]
            
            my_text = generated2.replace(to_remove,'')           
        
        return my_text


app.run_server(debug=True, use_reloader=False)

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on


100%|████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:05<00:00,  5.33s/it]
